In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow

In [3]:
# Read Image to be aligned
imgTest = cv2.imread('image.jpeg')

# Reference Reference image or Ideal image
imgRef = cv2.imread('reference.jpeg')

In [4]:
# imgTest = cv2.resize(imgTest, (1400, 1000))
# imgRef = cv2.resize(imgRef, (1400, 1000))

In [5]:
# Convert to grayscale.
imgTest_grey = cv2.cvtColor(imgTest, cv2.COLOR_BGR2GRAY)
imgRef_grey = cv2.cvtColor(imgRef, cv2.COLOR_BGR2GRAY)
height, width = imgRef_grey.shape

In [6]:
# Configure ORB feature detector Algorithm with 1000 features.
orb_detector = cv2.ORB_create(1000)

# Extract key points and descriptors for both images
keyPoint1, des1 = orb_detector.detectAndCompute(imgTest_grey, None)
keyPoint2, des2 = orb_detector.detectAndCompute(imgRef_grey, None)

# Display keypoints for reference image in green color
imgKp_Ref = cv2.drawKeypoints(imgRef, keyPoint1, 0, (0,222,0), None)
imgKp_Ref = cv2.resize(imgKp_Ref, (width//2, height//2))

cv2.imshow("Refer Img",imgKp_Ref)
# cv2.waitKey(0)

: 

In [9]:
# Match features between two images using Brute Force matcher with Hamming distance
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match the two sets of descriptors.
matches = matcher.match(des1, des2)
print(matches)

# Sort matches on the basis of their Hamming distance.
# matches.sort(key=lambda x: x.distance)
matches = sorted(matches, key = lambda x:x.distance)


# Take the top 90 % matches forward.
matches = matches[:int(len(matches) * 0.9)]
no_of_matches = len(matches)

# Display only 100 best matches {good[:100}
imgMatch = cv2.drawMatches(imgTest, keyPoint2, imgRef, keyPoint1, matches[:100], None, flags = 2)
imgMatch = cv2.resize(imgMatch, (width//3, height//3))

cv2.imshow("ImgMatch",imgMatch)
# cv2.waitKey(0)

(< cv2.DMatch 000001CE2AA6FE50>, < cv2.DMatch 000001CE2AA6FC50>, < cv2.DMatch 000001CE2C2F1630>, < cv2.DMatch 000001CE2C2F00D0>, < cv2.DMatch 000001CE2C2F0DF0>, < cv2.DMatch 000001CE2C2F18F0>, < cv2.DMatch 000001CE2C2F1BD0>, < cv2.DMatch 000001CE2C2F2A70>, < cv2.DMatch 000001CE2C2F26F0>, < cv2.DMatch 000001CE2C2F2490>, < cv2.DMatch 000001CE2C2F1930>, < cv2.DMatch 000001CE2C2F1850>, < cv2.DMatch 000001CE2C2F2930>, < cv2.DMatch 000001CE2C2F3DB0>, < cv2.DMatch 000001CE14639E30>, < cv2.DMatch 000001CE14639E70>, < cv2.DMatch 000001CE1463BD50>, < cv2.DMatch 000001CE2C5E71D0>, < cv2.DMatch 000001CE2C5E6770>, < cv2.DMatch 000001CE2C5E7E30>, < cv2.DMatch 000001CE2C5E5730>, < cv2.DMatch 000001CE2C5E7C10>, < cv2.DMatch 000001CE2C5E7F10>, < cv2.DMatch 000001CE2C5E7E50>, < cv2.DMatch 000001CE2C5E7CF0>, < cv2.DMatch 000001CE2C5E7D50>, < cv2.DMatch 000001CE2C5E7B30>, < cv2.DMatch 000001CE2C5E7AB0>, < cv2.DMatch 000001CE2C5E5A30>, < cv2.DMatch 000001CE2C5E7A30>, < cv2.DMatch 000001CE2C5E6450>, < cv2.D

In [10]:
# Define 2x2 empty matrices
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

# Storing values to the matrices
for i in range(len(matches)):
    p1[i, :] = keyPoint1[matches[i].queryIdx].pt
    p2[i, :] = keyPoint2[matches[i].trainIdx].pt

# Find the homography matrix.
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

In [11]:
# Use homography matrix to transform the unaligned image wrt the reference image.
aligned_img = cv2.warpPerspective(imgTest, homography, (width, height))

# Resizing the image to display in our screen (optional)
aligned_img = cv2.resize(aligned_img, (width//3, height//3))

# Copy of input image
imgTest_cp = imgTest.copy()
imgTest_cp = cv2.resize(imgTest_cp, (width//3, height//3))



In [12]:
cv2.imshow("Test Image",imgTest_cp)

In [13]:
cv2.imshow("Aligned IMg",aligned_img)